In [ ]:
pip install wikipedia-api

In [ ]:
############################### FROM HERE #####################################
import requests

def get_top_urls(topic, category=None, num_urls=5):
    base_url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "list": "search",
        "srsearch": topic,
        "srprop": "title",
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    urls = []
    for result in data.get('query', {}).get('search', []):
        title = result['title']
        article_url = f"https://en.wikipedia.org/wiki/{title.replace(' ', '_')}"
        if not category or category.lower() in title.lower():
            urls.append(article_url)

    return urls[:num_urls]

# Global variables to store the topic and top URLs
topic = ""
top_urls = []

def main():
    global topic, top_urls
    topic = input("Enter a topic: ")
    category = input("Enter a category (optional): ")
    top_urls = get_top_urls(topic, category)

    print(f"Top {len(top_urls)} URLs related to '{topic}' ({category if category else 'any category'}):")
    for i, url in enumerate(top_urls, start=1):
        print(f"{i}. {url}")

if __name__ == "__main__":
    main()


Enter a topic: Bollywood
Enter a category (optional): 
Top 5 URLs related to 'Bollywood' (any category):
1. https://en.wikipedia.org/wiki/Hindi_cinema
2. https://en.wikipedia.org/wiki/Lists_of_Hindi_films
3. https://en.wikipedia.org/wiki/Bollywood_Hungama
4. https://en.wikipedia.org/wiki/List_of_highest-grossing_Indian_films
5. https://en.wikipedia.org/wiki/Amitabh_Bachchan


In [ ]:
import requests
from bs4 import BeautifulSoup

google_urls=[]

def extract_summary_from_wikipedia(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    paragraphs = soup.find_all('p')
    summary = ' '.join(p.get_text() for p in paragraphs[:3])  # Extract the first 3 paragraphs as summary
    return summary

def get_top_google_urls(query, num_urls=5):
    base_url = "https://www.google.com/search"
    params = {
        "q": query,
        "num": num_urls
    }
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
    }

    try:
        response = requests.get(base_url, params=params, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        search_results = soup.find_all('div', class_='tF2Cxc')
        urls = [result.a['href'] for result in search_results if result.a]
        return urls
    except requests.RequestException as e:
        print("Error fetching data from Google Search:", e)
        return []

def main(topic, wikipedia_url, google_urls):
    print("Extracting summary from the Wikipedia article...")
    wikipedia_summary = extract_summary_from_wikipedia(wikipedia_url)

    print("Generating Google search query...")
    google_query = f"{topic} {wikipedia_summary}"

    print("Fetching additional URLs from Google...")
    new_google_urls = get_top_google_urls(google_query)
    google_urls.extend(new_google_urls)

    print(f"Top {len(new_google_urls)} URLs related to the Wikipedia article:")
    for i, url in enumerate(new_google_urls, start=1):
        print(f"{i}. {url}")

if __name__ == "__main__":
    # Provide the topic and Wikipedia URL stored from the previous cell
    topic = "Enter your topic here"
    wikipedia_urls = top_urls  # Assuming top_urls is a list of URLs
    for url in wikipedia_urls:
        main(topic, url, google_urls)


Extracting summary from the Wikipedia article...
Generating Google search query...
Fetching additional URLs from Google...
Top 5 URLs related to the Wikipedia article:
1. https://en.wikipedia.org/wiki/Hindi_cinema
2. https://www.quora.com/Why-are-Bollywood-movies-called-Bollywood-movies
3. https://batoolodeh.wordpress.com/bollywood/
4. https://en.wikipedia.org/wiki/Film_industry
5. https://artdepartmental.com/resources/filmmaking-glossary/
Extracting summary from the Wikipedia article...
Generating Google search query...
Fetching additional URLs from Google...
Top 5 URLs related to the Wikipedia article:
1. https://www.timeout.com/film/the-100-best-bollywood-movies-the-list
2. https://www.quora.com/How-many-Bollywood-films-are-produced-each-year
3. https://en.wikipedia.org/wiki/Hindi_cinema
4. https://m.imdb.com/list/ls064876977/
5. https://www.reddit.com/r/BollyBlindsNGossip/comments/wmidxu/the_downfall_of_bollywood_in_recent_times_what/
Extracting summary from the Wikipedia article..

In [ ]:
pip install wikipedia-api

In [ ]:
google_urls

['https://en.wikipedia.org/wiki/Hindi_cinema',
 'https://www.quora.com/Why-are-Bollywood-movies-called-Bollywood-movies',
 'https://batoolodeh.wordpress.com/bollywood/',
 'https://en.wikipedia.org/wiki/Film_industry',
 'https://artdepartmental.com/resources/filmmaking-glossary/',
 'https://www.timeout.com/film/the-100-best-bollywood-movies-the-list',
 'https://www.quora.com/How-many-Bollywood-films-are-produced-each-year',
 'https://en.wikipedia.org/wiki/Hindi_cinema',
 'https://m.imdb.com/list/ls064876977/',
 'https://www.reddit.com/r/BollyBlindsNGossip/comments/wmidxu/the_downfall_of_bollywood_in_recent_times_what/',
 'https://en.wikipedia.org/wiki/List_of_highest-grossing_Indian_films',
 'https://en.wikipedia.org/wiki/List_of_highest_domestic_net_collection_of_Hindi_films',
 'https://m.imdb.com/list/ls063771441/',
 'https://m.imdb.com/list/ls538172676/',
 'https://books.google.com/books?id=KAcAAAAAMBAJ&pg=PA2&lpg=PA2&dq=Enter+your+topic+here+%0A+%0A+This+is+a+ranking+of+the+highest-

In [ ]:
import requests
from bs4 import BeautifulSoup
import nltk
from collections import Counter

# Download NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

# Function to extract keywords using Named Entity Recognition (NER)
def extract_keywords_from_text(text):
    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Perform part-of-speech tagging
    pos_tags = nltk.pos_tag(tokens)

    # Perform named entity recognition (NER)
    named_entities = nltk.ne_chunk(pos_tags)

    # Extract named entities of type 'NE' with less than or equal to 6 words
    named_entities_list = []
    for chunk in named_entities:
        if hasattr(chunk, 'label'):
            entity = ' '.join(c[0] for c in chunk)
            if len(entity.split()) <= 6:  # Discard entities with more than 6 words
                named_entities_list.append(entity)

    return named_entities_list




# Function to extract content from URLs and perform keyword extraction
def extract_keywords_from_urls(urls):
    all_keywords = []
    for url in urls:
        try:
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            text = ' '.join([element.get_text() for element in soup.find_all(text=True)])
            keywords = extract_keywords_from_text(text)
            all_keywords.extend(keywords)
        except Exception as e:
            print(f"Error processing URL '{url}':", e)

    # Remove duplicates while preserving order
    unique_keywords = list(set(all_keywords))
    return unique_keywords


# Input: List of Google URLs
urls = google_urls



# Extract keywords from the provided URLs
unique_keywords = extract_keywords_from_urls(urls)

# Print or save unique keywords
print("Unique Keywords:", unique_keywords)

# Write unique keywords to a file
output_file = "unique_keywords.txt"
with open(output_file, 'w') as f:
    for keyword in unique_keywords:
        f.write(keyword + '\n')

print(f"Unique keywords saved to {output_file}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
<ipython-input-4-52149e4a61ba>:43: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text = ' '.join([element.get_text() for element in soup.find_all(text=True)])


Unique Keywords: ['Chaturbhuj Meher', 'Subhash', 'Santa Fe', 'Nizamuddin Dargah', 'Camera Film Festival', 'Ibomcha Sharma', 'Monroe', 'Roshan Kumari', 'Humpty Sharma Ki Dulhania', 'Hussain B.', 'Rajkummar Rao', 'Nouvelle Vague', 'Jeffrey Sachs Chandra Prasad', 'Hansal Mehta', 'PDF', 'NT Rama Rao Jr', 'Sobha Singh', 'Tinnu Anand', 'Collection', 'Glamour', 'Damayanti Joshi', 'Top Germany Movies', 'Good Films Azure', 'Bombay Dreams', 'Wimal Dissanayake', 'Ghosh', 'Richard', 'Mohammed Karim', 'Singam', 'Blank', 'Kanchan', 'Durga Khote', 'Pran Kishore Kaul', 'Morris', 'TeachAids', 'Allu Arjun', 'Telecom', 'Baswani', 'Paranjpye Deepak Parekh', 'Wikimedia Foundation', 'Economic', 'Academy Awards Winners', 'Shanti Hiranand', 'Hollywood Films', 'Part', 'Indiana University Press', 'Baipan Bhari Deva Box', 'DNA', 'Samyabrata Ray Goswami', 'Persian', 'Pakistani Qawwali', 'New Bollywood', 'Signature Amitabh Bachchan', 'Bombay Cinema', 'Daji Bhatawadekar', 'HAHK', 'Ratna Pathak Shah', 'Gustavo Santa

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering

# Load keywords from file
with open("unique_keywords.txt", "r") as file:
    keywords = file.readlines()

# Preprocess keywords: Remove newline characters
keywords = [keyword.strip() for keyword in keywords]

# Vectorize keywords using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(keywords)

# Cluster keywords using hierarchical clustering
num_clusters = 10  # You can adjust this based on your preference
agg_cluster = AgglomerativeClustering(n_clusters=num_clusters)
agg_cluster.fit(X.toarray())

# Create a dictionary to map clusters to keywords
cluster_keywords = {}
for i, label in enumerate(agg_cluster.labels_):
    cluster_keywords.setdefault(label, []).append(keywords[i])

# Print clusters and their keywords
for cluster, keywords in cluster_keywords.items():
    print(f"Cluster {cluster + 1}:")
    print(", ".join(keywords))
    print()


Cluster 1:
Chaturbhuj Meher, Subhash, Santa Fe, Nizamuddin Dargah, Ibomcha Sharma, Monroe, Roshan Kumari, Humpty Sharma Ki Dulhania, Hussain B., Rajkummar Rao, Nouvelle Vague, Jeffrey Sachs Chandra Prasad, Hansal Mehta, PDF, NT Rama Rao Jr, Tinnu Anand, Glamour, Damayanti Joshi, Top Germany Movies, Bombay Dreams, Wimal Dissanayake, Ghosh, Richard, Mohammed Karim, Singam, Blank, Kanchan, Durga Khote, Pran Kishore Kaul, Morris, TeachAids, Allu Arjun, Telecom, Baswani, Paranjpye Deepak Parekh, Wikimedia Foundation, Economic, Shanti Hiranand, Hollywood Films, Part, Indiana University Press, Baipan Bhari Deva Box, DNA, Samyabrata Ray Goswami, Persian, Pakistani Qawwali, New Bollywood, Bombay Cinema, Daji Bhatawadekar, HAHK, Ratna Pathak Shah, Gustavo Santaolalla, Pineapple Press, Kaho Naa, Sardaarji, Express News, CS1, Departures, Harvard University Asia Center, Criticism, Oil, Suriname, English, Yogendra Singh, Gennadi Mikhailovich Pechinkov, Hindi Dubbed Movies, Tijuana Brass Enterprises,

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering

# Load keywords from file
with open("unique_keywords.txt", "r") as file:
    keywords = file.readlines()

# Preprocess keywords: Remove newline characters
keywords = [keyword.strip() for keyword in keywords]

# Vectorize keywords using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(keywords)

# Cluster keywords using hierarchical clustering
num_clusters = 30  # You can adjust this based on your preference
agg_cluster = AgglomerativeClustering(n_clusters=num_clusters)
agg_cluster.fit(X.toarray())

# Create a dictionary to map clusters to keywords
cluster_keywords = {}
for i, label in enumerate(agg_cluster.labels_):
    cluster_keywords.setdefault(label, []).append(keywords[i])

# Print clusters and their keywords
for cluster, keywords in cluster_keywords.items():
    print(f"Cluster {cluster + 1}:")
    print(", ".join(keywords))
    print()

# Function to find the most significant words in each cluster
def get_cluster_names(vectorizer, cluster_keywords):
    feature_names = vectorizer.get_feature_names_out()
    cluster_names = {}

    for cluster, keywords in cluster_keywords.items():
        # Vectorize keywords in the cluster
        cluster_X = vectorizer.transform(keywords)
        # Sum TF-IDF scores for each term in the cluster
        tfidf_sum = cluster_X.sum(axis=0).A1  # Convert to 1D array
        # Get indices of the highest TF-IDF scores
        sorted_indices = np.argsort(tfidf_sum)[::-1]
        # Select top N words to represent the cluster
        top_n_words = [feature_names[index] for index in sorted_indices[:3]]
        cluster_names[cluster] = " ".join(top_n_words)

    return cluster_names

# Get cluster names
cluster_names = get_cluster_names(vectorizer, cluster_keywords)

# Print clusters with their names and keywords
for cluster, keywords in cluster_keywords.items():
    print(f"Cluster {cluster + 1} ({cluster_names[cluster]}):")
    print(", ".join(keywords))
    print()


Cluster 1:
Klaasen, Centre Insights Careers, Stay, Articles History, Ebooks, La Liga, Kumble, Maheesh, Boult, Kyle Jamieson, Micro Enterprises, Nathan, IFISA, Barclays, Conduct, MLS, Manipur, Brand Value Ranking League Table, Standard Suspect, Dirk Nannes, Insights Home Risk, Disney, Pat, Kar, Mahela Jayawardene, Achieve Massive Feat, Fruit, Hyderabad Ben Stokes, Captain, Aman Hakim, Americas Caribbean, Aditya, Pune, American Twenty20, Shimron Hetmyer Delhi Capitals Super Striker, Gap, Mark Boucher, Shamshuddin, Twenty20 Tournament, Nitin Menon, Daily Telegraph, Mayank Markande, Amit, Bose, Accept, Kane Williamson, Notes, Harshit, Guwahati, Chandra, Live Click, Memoir Artists, Footer, Siddarth Kaul, Ayushi, Hockey Players, Oceania Australian Women, Dillon, Brad Hogg, Russell, Rizwan, Interprovincial Twenty20, KKR Vs, Due, De Kock, Sharjah, Jos Buttler, Ref Brand, Affiliate Members, IPL Media Rights, Search Search, Singer Sonu Nigam, Changer, IPL Orange Cap, Slow, Pooran, Orange, Rachin

In [ ]:
pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from sentence_transformers import SentenceTransformer, util

# Load keywords from file
with open("unique_keywords.txt", "r") as file:
    keywords = file.readlines()

# Preprocess keywords: Remove newline characters
keywords = [keyword.strip() for keyword in keywords]

# Load a pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode keywords using the model
embeddings = model.encode(keywords)

# Cluster keywords using hierarchical clustering
num_clusters = 50  # You can adjust this based on your preference
agg_cluster = AgglomerativeClustering(n_clusters=num_clusters)
labels = agg_cluster.fit_predict(embeddings)

# Create a dictionary to map clusters to keywords
cluster_keywords = {}
for i, label in enumerate(labels):
    cluster_keywords.setdefault(label, []).append(keywords[i])

# Function to find the most representative words in each cluster
def get_cluster_names(cluster_keywords, model):
    cluster_names = {}

    for cluster, keywords in cluster_keywords.items():
        # Encode cluster keywords
        cluster_embeddings = model.encode(keywords)
        # Compute the centroid of the cluster
        centroid = np.mean(cluster_embeddings, axis=0)
        # Compute cosine similarities between centroid and each keyword embedding
        similarities = util.pytorch_cos_sim(centroid, cluster_embeddings)[0].numpy()
        # Get index of the highest similarity
        most_representative_index = np.argmax(similarities)
        # Select the most representative word for the cluster
        most_representative_word = keywords[most_representative_index]

        # Attempt to generate a concise name (1-2 words)
        cluster_name_words = most_representative_word.split()[:2]
        cluster_name = " ".join(cluster_name_words)
        cluster_names[cluster] = cluster_name

    return cluster_names

# Get cluster names
cluster_names = get_cluster_names(cluster_keywords, model)

# Print clusters with their names and keywords
for cluster, keywords in cluster_keywords.items():
    print(f"Cluster {cluster + 1} ({cluster_names[cluster]}):")
    print(", ".join(keywords))
    print()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Cluster 17 (Mukesh Mehra):
Chaturbhuj Meher, Subhash, Hansal Mehta, Damayanti Joshi, Nasreen Munni, Ketan Mehta, Lage Raho Munna Bhai, Gayatri Joshi, Shree Lal Joshi, Subhash K Jha, Cheez Badi Hai Mast Mast, Igwe, Murad, Jawaani Hai Deewani, Mukerji, Ismat Chughtai, Meenakshi Seshadri, Meenakshi, Mehmood Dhaulpuri, Raho Munna Bhai, Manoj Joshi, Janeshwar Mishra Succeeded, Mishra, Johny Mera Naam Mehboob Studio, Hain, Munna, Mukesh Mehra, Meena Kumari Genre, Sudhir Mishra, Banda Yeh Bindaas Hai, Bhimsen Joshi, Meher Vij, Jumma Chumma, Meenakshi Chitharanjan, Shivnath Mishra, Yeh Jawaani Hai Deewani, Haiya, Iyengar F., Subhash K, Mukh O Mukhosh, Sarita Joshi, Mehmood, Munni, Charles Igwe, Hai, Yeh Jawaani Hai Deewani Box, Chupke Chupke, Sadhu Meher, Mekemson, Sohrab Modi, Hum Aapke Hain, Vijaya Mehta, Govind Nihalaniand Vijaya Mehta, Raj Mehra, Prasoon Joshi, Meena Iyer, Mehboob Studio, Mursi, Karma Subhash Ghai, Kohram, Meena Kumari, Sisir Mishra, Sooryavansham, Hai Sub, Meheli, Mukesh,